<a href="https://colab.research.google.com/github/arvindhvasu/GL_Pneumonia_Detection/blob/Dev_Arvindh/model_Resnet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%tensorflow_version 2.x

In [188]:
import tensorflow
tensorflow.__version__

'2.2.0'

In [0]:
import pandas as pd 
import numpy as np
from seaborn import countplot
from matplotlib.pyplot import figure, show
import os
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import seaborn as sns

In [190]:
!pip3 install pydicom
import pydicom
import os
from skimage.transform import resize
import keras
import pandas as pd
import pickle
import cv2

In [0]:
from keras.optimizers import Adam
from keras.models import Sequential, Model, load_model
from keras.applications.resnet50 import ResNet50
from keras.layers import Input, Flatten, Dense, GlobalAveragePooling2D, Conv2D
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from PIL import Image

#Mount Google Drive

In [192]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#Set your project path 
project_path =  'drive/My Drive/Colab Notebooks/Capstone'
train_img_path = 'drive/My Drive/Colab Notebooks/Capstone/dataset/stage_2_train_images'
valid_img_path = 'drive/My Drive/Colab Notebooks/Capstone/dataset/stage_2_train_images'
test_img_path = 'drive/My Drive/Colab Notebooks/Capstone/dataset/stage_2_test_images'

In [0]:
train_df_pick = "/train_df_pickle"
valid_df_pick = "/valid_df_pickle"

# we open the file for reading
fileObject = open(project_path+train_df_pick,'rb')  
fileObject1 = open(project_path+valid_df_pick,'rb')  

# load the object from the file into temp var b
train_df = pickle.load(fileObject)  
valid_df = pickle.load(fileObject1) 

In [195]:
train_patient_Ids = train_df.patientId.unique()
print("The number of unique patient id in the training dataset is "+ str(train_patient_Ids.shape[0]))

The number of unique patient id in the training dataset is 21908


In [196]:
valid_patient_Ids = valid_df.patientId.unique()
print("The number of unique patient id in the validation dataset is "+ str(valid_patient_Ids.shape[0]))

The number of unique patient id in the validation dataset is 5914


In [0]:
train_df['Path'] = train_df.apply(lambda row: train_img_path + '/' +  row.patientId + '.dcm', axis = 1) 
valid_df['Path'] = valid_df.apply(lambda row: train_img_path + '/' +  row.patientId + '.dcm', axis = 1) 

In [0]:
train_df['Class_Cat'] = train_df.apply(lambda row: 'NORMAL' if row.Target == 0 else 'PNEUMONIA', axis = 1) 
valid_df['Class_Cat'] = valid_df.apply(lambda row: 'NORMAL' if row.Target == 0 else 'PNEUMONIA', axis = 1) 

In [0]:
sample_Train = train_df.sample(3000)
sample_Valid = train_df.sample(1500)

In [0]:
import shutil, os
for f in sample_Valid.iterrows():
    if f[1].Target == 1:
        shutil.copy(f[1].Path, 'drive/My Drive/Colab Notebooks/Capstone/dataset_new/Valid/PNEUMONIA')
    else:
        shutil.copy(f[1].Path, 'drive/My Drive/Colab Notebooks/Capstone/dataset_new/Valid/NORMAL')
    

In [0]:
train_df.head()

,patientId,x,y,width,height,Target,class,PatientAge,PatientSex,ViewPosition,Path,Class_Cat
27976,c7471748-599f-4cf3-bd8f-3913c24ee6ea,NaN,NaN,NaN,NaN,0,No Lung Opacity / Not Normal,63,F,PA,drive/My Drive/Colab Notebooks/Capstone/datase...,Normal
21827,a87fca97-b625-46c0-a9af-9456c03aaaaf,232.0,214.0,226.0,491.0,1,Lung Opacity,40,M,AP,drive/My Drive/Colab Notebooks/Capstone/datase...,Pneumonia
31530,e1aa3281-83f1-43f3-908d-d32ee4f032a8,NaN,NaN,NaN,NaN,0,No Lung Opacity / Not Normal,76,M,AP,drive/My Drive/Colab Notebooks/Capstone/datase...,Normal
20496,a006bc5e-87c6-48ca-92ac-32cf22ce5a86,NaN,NaN,NaN,NaN,0,No Lung Opacity / Not Normal,47,M,PA,drive/My Drive/Colab Notebooks/Capstone/datase...,Normal
28420,ca61ab53-b836-451e-b57a-3a8197a8b5ef,NaN,NaN,NaN,NaN,0,No Lung Opacity / Not Normal,46,M,AP,drive/My Drive/Colab Notebooks/Capstone/datase...,Normal


#Custom Data generator

In [0]:
class Generator(keras.utils.Sequence):
  def __init__(self, unique_patient_ids, folder, dataframe, batch_size=100, shuffle=False, image_size=256, predict=False):
        self.unique_patient_ids = unique_patient_ids
        self.folder = folder
        self.dataframe = dataframe
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.image_size = image_size
        self.predict = predict
        self.on_epoch_end()
   
  def __load__(self, pat_id):
        patIdWithExt = pat_id+'.dcm'
        # load dicom file from pixel array
        img = pydicom.dcmread(os.path.join(self.folder, patIdWithExt)).pixel_array
        # create empty array
        msk = np.zeros(img.shape) 
        
        if pat_id in self.dataframe["patientId"].values:
            pat_info = self.dataframe[self.dataframe.patientId == pat_id]

            # loop through patient rows in the df
            for info in pat_info.iterrows():
                row = info[1]
                if row.Target == 1:
                    x = int(row.x)
                    y = int(row.y)
                    msk[y: int(row.y) + int(row.height), x: int(row.x) + int(row.width)] = 1 
        
        img = cv2.resize(img, (self.image_size, self.image_size))
        msk = cv2.resize(msk, (self.image_size, self.image_size))
        img = np.expand_dims(img, -1)
        msk = np.expand_dims(msk, -1)   

        image_RGB = np.ndarray(shape=(img.shape[0], img.shape[1], 3), dtype= np.uint8)
        image_RGB[:, :, 0] = img[:, :, 0]
        image_RGB[:, :, 1] = img[:, :, 0]
        
        msk_RGB = np.ndarray(shape=(msk.shape[0], msk.shape[1], 3), dtype= np.uint8)
        msk_RGB[:, :, 0] = msk[:, :, 0]
        msk_RGB[:, :, 1] = msk[:, :, 0]
        
        return image_RGB, msk_RGB
        '''
        img2 = np.concatenate((img,)*3, axis=-1) 
        msk2 = np.concatenate((msk,)*3, axis=-1)
        return img2, msk2
        '''

  def __loadpredict__(self, pat_id):
        patIdWithExt = pat_id
        # load dicom file from pixel array
        img = pydicom.dcmread(os.path.join(self.folder, patIdWithExt)).pixel_array
        # resize image
        img = resize(img, (self.image_size, self.image_size), mode='reflect')
        # add trailing channel dimension
        img = np.expand_dims(img, -1)
        return img

  def __getitem__(self, index):
        if self.predict:
            # load files
            imgs = [self.__loadpredict__(pat_id_row) for pat_id_row in self.unique_patient_ids]
            # create numpy batch
            imgs = np.array(imgs)
            return imgs, self.unique_patient_ids
        else:    
            # select pat_ids by batches
            batch_pat_ids = self.unique_patient_ids[index*self.batch_size : (index+1)*self.batch_size]

            # load pat ids by loop
            items = [self.__load__(pat_id_row) for pat_id_row in batch_pat_ids]
            imgs, msks = zip(*items)

            # create numpy batch
            imgs = np.array(imgs)
            msks = np.array(msks)
        return imgs, msks

  def __len__(self):
        if self.predict:
            return int(np.ceil(len(self.unique_patient_ids) / self.batch_size))
        else:
            # __len__ provides number of batches per epoch
            return int(np.floor(self.unique_patient_ids.shape[0] / self.batch_size))

  def on_epoch_end(self):
        if self.predict:
            self.indexes = np.arange(len(self.unique_patient_ids))
        else:
            self.indexes = np.arange(self.unique_patient_ids.shape[0])
        if self.shuffle == True:
              np.random.shuffle(self.indexes)

In [0]:
valid_df.sample(3)["Target"].values

array([0, 0, 1])

In [0]:
from keras.applications.resnet50 import ResNet50 as PTModel, preprocess_input
from keras_preprocessing.image import ImageDataGenerator
img_gen_args = dict(samplewise_center=False, 
                              samplewise_std_normalization=False, 
                              horizontal_flip = True, 
                              vertical_flip = False, 
                              height_shift_range = 0.05, 
                              width_shift_range = 0.02, 
                              rotation_range = 3, 
                              shear_range = 0.01,
                              fill_mode = 'nearest',
                              zoom_range = 0.05,
                              preprocessing_function=preprocess_input)
img_gen = ImageDataGenerator(**img_gen_args)

In [202]:
train_generator=img_gen.flow_from_directory(
        'drive/My Drive/Colab Notebooks/Capstone/dataset_new/Train/NORMAL',
        target_size=(224, 224),
        batch_size=32,
        
        class_mode='binary')

Found 0 images belonging to 1 classes.


In [0]:
len(train_generator)

0

In [0]:
train_gen = Generator(train_patient_Ids[:300], train_img_path, train_df, batch_size=32, shuffle=False, image_size=224)
valid_gen = Generator(valid_patient_Ids[:150], valid_img_path, valid_df, batch_size=32, shuffle=False, image_size=224)

In [0]:
IMAGE_SIZE = [224,224]
base_resnet = ResNet50(input_shape=IMAGE_SIZE +[3], weights='imagenet', include_top=False)

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [0]:
for layer in base_resnet.layers:
  layer.trainable = False

result = GlobalAveragePooling2D()(base_resnet.output)
predictions = Dense(2, activation='sigmoid')(result)

In [0]:
model = Model(inputs=base_resnet.input, outputs=predictions)
model.compile(optimizer='adam',loss='binary_crossentropy',
              metrics=['accuracy', tensorflow.keras.metrics.MeanIoU(num_classes=2)])

checkpoint = ModelCheckpoint("model_Resnet.h5", monitor="val_loss", verbose=1, save_best_only=True,
                             save_weights_only=True, mode="auto", period=1)
stop = EarlyStopping(monitor="val_loss", patience=10, mode="auto")
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=5, min_lr=1e-6, verbose=1, mode="auto")
model.summary()

Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_10[0][0]                   
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
____________________________________________________________________________________________

In [0]:
history = model.fit_generator(train_gen, 
                              validation_data=valid_gen, 
                              callbacks=[checkpoint, reduce_lr, stop],
                              epochs=4, 
                              workers=1, 
                              steps_per_epoch=len(train_gen),
                              validation_steps= len(valid_gen),
                              use_multiprocessing=False)

Epoch 1/4


ValueError: ignored